In [10]:
# list of candidates in the order listed on the voting site
candidates = ["Alexandre Eichenberger (IBM)",	"Andreas Fehlner (TRUMPF Laser GmbH)",	"Mayank Kaushik (NVIDIA)",	"Amira Moussa (CheetahAI)",	"Rajeev Nalawadi (Rajeev Nalawadi)",	"Prasanth Pulavarthi (Microsoft)",	"Egor Pushkin (Oracle Cloud Infrastructure"]

# votes with company (added manually), name (replaced manually from email address, which should be avoided in published docs), and votes ranking for each candidate
votes = [
["MS", "Nathaniel McVicar",	1,	6,	4,	7,	3,	2,	5],
["IBM", "Alex Eichenberger",	2,	6,	3,	6,	3,	1,	5],
["NVIDIA", "Joaquin Anton",	3,	4,	1,	7,	5,	2,	6],
["IBM", "Haruki Imai",	1,	7,	3,	7,	3,	2,	5],
["NVIDIA", "Kevin Chen",	3,	5,	1,	5,	4,	2,	5],
["NVIDIA", "Rajeev Rao",	3,	6,	1,	7,	4,	2,	5],
["MS", "G. Ramalingam",	2,	6,	4,	7,	3,	1,	5],
["NVIDIA", "Zhenhua Wang",	3,	6,	2,	7,	4,	1,	5],
["IBM", "Ettore Tiotto",	1,	7,	2,	7,	2,	2,	7],
["MS", "Wei-Sheng Chin",	2,	7,	5,	3,	4,	1,	6],
["MS", "Prasanth Pulavarthi",	1,	4,	1,	4,	1,	1,	2],
["MS", "Gary Miguel",	2,	5,	4,	7,	3,	1,	6],
["IBM", "Yasushi Negishi",	1,	7,	7,	7,	7,	7,	7],
["MS", "Hariharan Seshadri",	3,	7,	2,	5,	6,	1,	4],
["MS", "Chun-Wei (Jacky) Chen",	2,	7,	4,	7,	3,	1,	7],
["IBM", "Chin Huang",	1,	7,	1,	7,	1,	1,	7],
["IBM", "Degao Chu",	1,	7,	2,	5,	4,	2,	7],
["MS", "Aishwarya Bhandare",	3,	5,	2,	7,	7,	1,	4],
["IBM", "Kevin O'Brien",	1,	7,	3,	7,	3,	2,	3],
["MIT", "Tian Jin",	1,	7,	7,	7,	7,	7,	7],
["IBM", "Tung Le",	1,	3,	1,	2,	1,	1,	2],
["MS", "Yufeng Li",	5,	7,	2,	6,	4,	1,	3],
["VMWARE", "Anna Jung",	2,	2,	1,	7,	1,	1,	6],
["MS", "Faith Xu",	5,	7,	2,	6,	3,	1,	4],
["IBM", "Gheorghe-Teodor Bercea",	1,	2,	7,	6,	5,	3,	4],
["INTEL", "Rajeev Nalawadi",	3,	7,	4,	7,	1,	2,	7],
["IBM", "Whitney Tsang",	1,	7,	2,	7,	2,	3,	7],
["MS", "Liqun Fu",	2,	6,	3,	7,	4,	1,	5],
["PreferredNetworks", "Shinichiro Hamaji",	3,	5,	1,	7,	6,	2,	4],
["INTEL", "Michał Karzyński",	3,	7,	7,	7,	1,	2,	7],
["MS", "Dmitri Smirnov",	3,	4,	2,	4,	5,	1,	7],
["IBM", "Tong Chen",	1,	7,	7,	7,	7,	7,	7],
["ALIBABA", "Ke Zhang", 1, 2, 3, 7, 4, 5, 7] # voted by email from china
]

In [11]:
import numpy as np

# from the list of all ballots, seclect the ones for this company (or all if company is "ALL")
def ballot_for_company(ballots, select_company, verbose=0):
    res = []
    for b in ballots:
        curr_company, curr_ballot = b[0], b[2:]
        if select_company == 'ALL' or curr_company == select_company:
            res.append(curr_ballot)
            if verbose:
                print(curr_company, curr_ballot)
    return np.array(res)

# create a new ballot reflecting the choices in the given ballot sample
def rank(ballots, candidates, verbose=0):
    cnum = candidates.size
    #m[a,b] is "time folks preferred candidate b over a"
    m = np.zeros((cnum, cnum), dtype=int)
    for b in ballots:
        for j in range(cnum):
            for k in range(cnum):
                if b[k]>b[j]:
                    m[k][j] = m[k][j] + 1
    total = np.sum(m, axis=0) # sum over columns
    # (candidate id, preference score) ranked by most preferred first
    rank = sorted(zip(range(cnum),total), reverse=True, key=lambda record: record[1])
    # new ballot: where candidates that have the same preferrence have the same score (low is better)
    new_ballot = np.zeros(cnum, dtype=int)
    preference = 0
    previous_score = 10000000
    for p in range(cnum): # by order of decreasing popularity
        curr_candidate_id, curr_candidate_score = rank[p]
        if curr_candidate_score != previous_score:
            preference +=1 # different score, advance by 1
        new_ballot[curr_candidate_id] = preference
        previous_score = curr_candidate_score   
    if verbose:
        print("Matrix m[a, b], times voters preferred candidate b over a:")
        for i in range(cnum):
            print(i, ": ", m[i])
        print("Totals (sum over columns of m) =", total)
        print("Rank (candidate id, candidate preference score) = ", rank)
        print("New ballot = ", new_ballot)
    return new_ballot

# create a ballot that reflects all the votes per company
def rank_per_company(votes, candidates, verbose=0):
    companies = np.unique(votes[:, 0])
    ballot_per_company = []
    for com in companies:
        print("Ballot for company = ", com)
        new_ballot = rank(ballot_for_company(votes, com), candidates, verbose=verbose)
        print("  is ", new_ballot)
        ballot_per_company.append(new_ballot)
        
    final_ballot = rank(np.array(ballot_per_company), candidates, verbose=verbose)
    if verbose:
        print("Final ballot = ", final_ballot)
    return final_ballot

# translate a ballot into the list of ordered candidates using names
def ballot_to_candidates(ballot, candidates):
    cnum = candidates.size
    # (candidate id, order) ranked by most preferred first
    rank = sorted(zip(range(cnum),ballot), key=lambda record: record[1])
    for r in rank:
        print(r[1], ":", candidates[r[0]])

In [12]:
# use numpy arrays
vv = np.array(votes)
cc = np.array(candidates)

#ballot_for_company(vv, 'INTEL')
#rank(ballot_for_company(vv, "INTEL"), cc, verbose=1)

# actual vote
final_ballot_per_company = rank_per_company(vv, cc)
print("\nFinal ballot (summarizing per company)=", final_ballot_per_company)
ballot_to_candidates(final_ballot_per_company, cc)

# control vote
ballot_per_voter = rank(ballot_for_company(vv, "ALL"), cc)
print("\nBallot per voter =", ballot_per_voter)
ballot_to_candidates(ballot_per_voter, cc)

Ballot for company =  ALIBABA
  is  [1 2 3 6 4 5 6]
Ballot for company =  IBM
  is  [1 6 4 7 3 2 5]
Ballot for company =  INTEL
  is  [3 5 4 5 1 2 5]
Ballot for company =  MIT
  is  [1 2 2 2 2 2 2]
Ballot for company =  MS
  is  [2 7 3 6 4 1 5]
Ballot for company =  NVIDIA
  is  [3 5 1 6 4 2 5]
Ballot for company =  PreferredNetworks
  is  [3 5 1 7 6 2 4]
Ballot for company =  VMWARE
  is  [2 2 1 4 1 1 3]

Final ballot (summarizing per company)= [1 5 3 7 4 2 6]
1 : Alexandre Eichenberger (IBM)
2 : Prasanth Pulavarthi (Microsoft)
3 : Mayank Kaushik (NVIDIA)
4 : Rajeev Nalawadi (Rajeev Nalawadi)
5 : Andreas Fehlner (TRUMPF Laser GmbH)
6 : Egor Pushkin (Oracle Cloud Infrastructure
7 : Amira Moussa (CheetahAI)

Ballot per voter = [1 6 3 7 4 2 5]
1 : Alexandre Eichenberger (IBM)
2 : Prasanth Pulavarthi (Microsoft)
3 : Mayank Kaushik (NVIDIA)
4 : Rajeev Nalawadi (Rajeev Nalawadi)
5 : Egor Pushkin (Oracle Cloud Infrastructure
6 : Andreas Fehlner (TRUMPF Laser GmbH)
7 : Amira Moussa (CheetahAI